<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [95]:
import pandas as pd
import psycopg2
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [96]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [97]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [98]:
# текст запроса
query_3_1 = f'''
SELECT 
    COUNT(*)  "Count_vacancies" 
FROM
    vacancies
'''

In [99]:
# результат запроса
df_vacanies = pd.read_sql_query(query_3_1, connection)
df_vacanies

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\2311803058.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vacanies = pd.read_sql_query(query_3_1, connection)


,Count_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [100]:
# текст запроса
query_3_2 = f'''
SELECT 
    COUNT(*) as "Count_employers"
FROM
    employers
'''

In [101]:
# результат запроса
df_employers = pd.read_sql_query(query_3_2, connection)
df_employers

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\3337367508.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employers = pd.read_sql_query(query_3_2, connection)


,Count_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [102]:
# текст запроса
query_3_3 = f'''
SELECT 
    COUNT(*) as "Count_regions"
FROM 
    areas
'''

In [103]:
# результат запроса
df_areas = pd.read_sql_query(query_3_3, connection)
df_areas

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\2360458281.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_areas = pd.read_sql_query(query_3_3, connection)


,Count_regions
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [104]:
# текст запроса
query_3_4 = f'''
SELECT 
    COUNT(*) as "Count_fields_activity"
FROM 
    industries
'''

In [105]:
# результат запроса
df_industries = pd.read_sql_query(query_3_4, connection)
df_industries

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\1986887719.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_industries = pd.read_sql_query(query_3_4, connection)


,Count_fields_activity
0,294


***

Выводы по предварительному анализу данных:

* В базе содержится 49197  вакансий
* География представлена 1362 городами
* 23501 работателей, которые представлены  294 сферами деятельности 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [106]:
# текст запроса
n = 5 # Количество  первых строк, которое вывведит на  экран
query_4_1 = f'''
SELECT 
    a.name as "Region",
    COUNT(v.id) as "Count_vacancies"
FROM 
    vacancies v
JOIN areas a on v.area_id = a.id      
GROUP BY a.id
ORDER BY count(v.id) DESC
LIMIT {n}     
'''

In [107]:
# результат запроса
df_cntarea = pd.read_sql_query(query_4_1, connection)
df_cntarea

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\3795629963.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cntarea = pd.read_sql_query(query_4_1, connection)


,Region,Count_vacancies
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [108]:
# текст запроса
query_4_2 = f'''
SELECT
    COUNT(v.id) as "Count_vacancies with salary"
FROM 
    vacancies v
WHERE  v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL      
'''

In [109]:
# результат запроса
df_salary = pd.read_sql_query(query_4_2, connection)
df_salary

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\3578541044.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_salary = pd.read_sql_query(query_4_2, connection)


,Count_vacancies with salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [110]:
# текст запроса
query_4_3 = f'''
SELECT
    ROUND(AVG(v.salary_from)) as "Salary_from",
    ROUND(AVG(v.salary_to)) as "Salary_to"
FROM 
    vacancies v
      
'''

In [111]:
# результат запроса
salary_difference = pd.read_sql_query(query_4_3, connection)
salary_difference

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\2184446509.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_difference = pd.read_sql_query(query_4_3, connection)


,Salary_from,Salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [112]:
# текст запроса
query_4_4 = f'''
SELECT
    v.schedule as "Schedule",
    v.employment as "Employment",
    COUNT(v.id)  as " Count_vacancies schedule - employment"
FROM 
    vacancies v
GROUP BY v.schedule, v.employment
ORDER BY COUNT(v.id) DESC      
'''

In [113]:
# результат запроса
schedule_employment = pd.read_sql_query(query_4_4, connection)
schedule_employment

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\1022405654.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schedule_employment = pd.read_sql_query(query_4_4, connection)


,Schedule,Employment,Count_vacancies schedule - employment
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [114]:
# текст запроса
query_4_5 = f'''
SELECT
    v.experience as "Experience",
    COUNT(v.id)  as " Count"
FROM 
    vacancies v
GROUP BY v.experience
ORDER BY COUNT(v.id)      
'''

In [115]:
# результат запроса
df_experience = pd.read_sql_query(query_4_5, connection)
df_experience

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\1343279108.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_experience = pd.read_sql_query(query_4_5, connection)


,Experience,Count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий:
* Количество вакансий, где  указана заработная плата составляет 24073. Можно сделать вывод,  что оплата труда будет  определяться при собеседовании и оценки навыков соискателя работадателем. Основная часть работадателей ищут работников с минимальным опытом работы или без опыта предполагает,  что компании готовы вкладываться в развитие своих новых подчененых возможно за счет минимизации их зарплаты.
* Пятерка городов по   количеству  вакансий представлена крупными городами России, Белорусии и Казахстана.
* В основном ещут работников на полную занятость. Преобладают вакансии  Полный день - 	Полная занятость.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [116]:
# текст запроса
query_5_1 = f'''
(SELECT
    e.name as "Employer_name",
    COUNT(v.id) as "Count_employer"
FROM 
    vacancies v
JOIN employers e ON e.id = v.employer_id
GROUP BY e.id
ORDER BY COUNT(v.id) DESC
LIMIT 1)

UNION ALL

(SELECT
    e.name as "Employer_name",
    COUNT(v.id) as "Count_employer"
FROM 
    vacancies v
JOIN employers e ON e.id = v.employer_id
GROUP BY e.id
ORDER BY COUNT(v.id) DESC
OFFSET 4 LIMIT 1)
'''

In [117]:
# результат запроса
df_employers = pd.read_sql_query(query_5_1, connection)
df_employers

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\3264855683.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employers = pd.read_sql_query(query_5_1, connection)


,Employer_name,Count_employer
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [118]:
# текст запроса
query_5_2 = f'''
SELECT 
    a.name as "Name_area",
    COUNT(e.name) as "Count_employers",
    COUNT(v.id) as "Count_vacancies"
FROM 
    employers e
LEFT JOIN vacancies v ON e.area = v.area_id
JOIN areas a ON a.id  = e.area
GROUP BY a.name
ORDER BY COUNT(v.id) ASC, COUNT(e.name) DESC

'''

In [119]:
# результат запроса
df_areas_cntvacancies = pd.read_sql_query(query_5_2, connection)
df_areas_cntvacancies

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\24889963.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_areas_cntvacancies = pd.read_sql_query(query_5_2, connection)


,Name_area,Count_employers,Count_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
649,Новосибирск,1149438,1149438
650,Алматы,1364132,1364132
651,Минск,2354880,2354880
652,Санкт-Петербург,6320667,6320667


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [120]:
# текст запроса
query_5_3 = f'''
SELECT 
    DISTINCT e.name as "Name_employer",
    COUNT(DISTINCT v.area_id) as "Count_regions"
FROM 
    employers e
JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.name
ORDER BY  COUNT(DISTINCT v.area_id) DESC   
'''

In [121]:
# результат запроса
df_area_employers = pd.read_sql_query(query_5_3, connection)
df_area_employers

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\497139089.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_area_employers = pd.read_sql_query(query_5_3, connection)


,Name_employer,Count_regions
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,Яроцкий Е. В./ MoonX,1
14762,Ясли-сад №374 г. Минска,1
14763,ЯСП,1
14764,Ястро-Переработка,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [122]:
# текст запроса

query_5_4 = f'''
SELECT
    COUNT(*) as "Count_employer"
FROM 
    employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL 
'''

In [123]:
# результат запроса
df_employers_industries = pd.read_sql_query(query_5_4, connection)
df_employers_industries

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\2964056463.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employers_industries = pd.read_sql_query(query_5_4, connection)


,Count_employer
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [124]:
# текст запроса
query_5_5 = f'''
SELECT
    e.name as "Name_employer",
    COUNT(ei.industry_id) as "Count_industries"
FROM 
    employers e
JOIN employers_industries ei ON e.id = ei.employer_id
GROUP BY e.name
HAVING COUNT(ei.industry_id) = 4
ORDER BY e.name
OFFSET 2 LIMIT 1 
'''

In [125]:
# результат запроса
df_cnt_industries = pd.read_sql_query(query_5_5, connection)
df_cnt_industries

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\457072743.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cnt_industries = pd.read_sql_query(query_5_5, connection)


,Name_employer,Count_industries
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [126]:
# текст запроса
query_5_6 = f'''
SELECT
    COUNT(*) as "Count_employers"
FROM 
    industries i
JOIN employers_industries ei ON i.id = ei.industry_id
WHERE i.name = 'Разработка программного обеспечения' 
'''


In [127]:
# результат запроса
df_employer_software_development = pd.read_sql_query(query_5_6, connection)
df_employer_software_development

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\3566810492.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employer_software_development = pd.read_sql_query(query_5_6, connection)


,Count_employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [155]:
# код для получения списка городов-милионников
# Получаем dataset городов-милионников
df_cities = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')[1]
# Извлекаем из  базы список городов
cities = tuple(df_cities['Город'])


In [162]:
# текст запроса
query_5_7 = f'''
SELECT
    a.name as name_city,
    COUNT(v.id) as count_vacancies
FROM 
    vacancies v
JOIN employers e ON e.id = v.employer_id
JOIN areas a   ON a.id = v.area_id
WHERE a.name IN {cities} AND e.name = 'Яндекс'
GROUP BY a.name 

UNION ALL

SELECT
    'Total' ,COUNT(v.id)
FROM 
    vacancies v
JOIN employers e ON e.id = v.employer_id
JOIN areas a   ON a.id = v.area_id
WHERE a.name IN {cities} AND e.name = 'Яндекс'
ORDER BY 2
'''

In [163]:
# результат запроса
df_yandex = pd.read_sql_query(query_5_7, connection)
df_yandex

C:\Users\johno\AppData\Local\Temp\ipykernel_3656\4049419416.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_yandex = pd.read_sql_query(query_5_7, connection)


,name_city,count_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Выводы по анализу работодателей:

* Большое количество вакансий представлены компаниями IT и банковских  сферах. Также крупные компании ищут работниках во многих регионах, что говорит о развитой филиальной  сети.
* Многие компании имеют несколько сфер  деятельности , что говорит  о развитии в нескольких направлениях и  поиска работников разных специальностей.
* 3553 компаний занимаются разработкой программного обеспечения
* У одного из лидеров IT сфере компания "Яндекс" представлены вакансии во всех крупных городах России. Можно предположить о постаянном росте компании или нехватки специалистов.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [132]:
# текст запроса

In [133]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [134]:
# текст запроса

In [135]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [136]:
# текст запроса

In [137]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [138]:
# текст запроса

In [139]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [140]:
# текст запроса

In [141]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [142]:
# текст запроса

In [143]:
# результат запроса

***

In [144]:
# выводы по предметному анализу

# Общий вывод по проекту

In [145]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования